<div class="alert alert-block alert-info">
    <h1>Thiago Kuma</h1>
</div>

http://linkedin.com/in/thiagokuma

In [ ]:
import pandas as pd
import numpy as np
import datetime

import seaborn as sns
import matplotlib.pyplot as plt

<div class="alert alert-block alert-info">
    <h1>Exploração de Dados</h1>
</div>

## Objetivo

### Entendimento do que é o dataset
    - Observar características
        - Tamanho
        - Completude
        - Retidão ("Corretude")
        - Tipos de dados
        - Possível relações entre os dados
    - Passos importantes
        - Carregamento
        - Visualização
        - Manipulação
        - Correlação

<div class="alert alert-block alert-info">
    <h2>Fontes dos dados</h2>
</div>

### Dados de Imigrantes
http://dados.prefeitura.sp.gov.br/pt_PT/dataset/base-de-dados-do-centro-de-referencia-e-atendimento-para-imigrantes-crai/resource/7eb866c0-02a3-4215-893d-d0b62a196c0c

<div class="alert alert-block alert-info">
    <h2>Carregamento de Dados</h2>
</div>

In [ ]:
!ls

### Em geral existe um dicionário de dados

In [ ]:
!head -n 15 dicionario-de-bancocrai2014a2019.csv

In [ ]:
pd.read_csv('dicionario-de-bancocrai2014a2019.csv', sep=';').head()

### Observação inicial dos dados

#### Fatos importantes na observação
1) O arquivo é CSV  
2) Existe coluna de cabeçalho  
3) As colunas estão separadas por ";"  
4) O encoding do arquivo

In [ ]:
!head -n 2  bancocrai2014a2019---sistematizacao-geoinfo-atualizada.csv

##### Encoding em Python
https://docs.python.org/3/library/codecs.html#standard-encodings

In [ ]:
df = pd.read_csv('bancocrai2014a2019---sistematizacao-geoinfo-atualizada.csv', sep=';', encoding='iso8859_2')
df.tail()

<div class="alert alert-block alert-info">
    <h2>Observação Dados</h2>
</div>

In [ ]:
df.shape #Quantidade linhas x colunas

In [ ]:
df.columns #Colunas nomes

In [ ]:
df.info() #Tipo e contagem por coluna

In [ ]:
df['sabendo_crai'].unique()

<div class="alert alert-block alert-info">
    <h2>Transformações Dados</h2>
</div>

In [ ]:
df['data_entrada_brasil'].unique()

### Transformando em data

https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
datetime.datetime.strptime('16/06/2015', '%d/%m/%Y')

##### O erro!!! Vamos então aplicar transformando string em data :D
- Dataframe.apply / lambda https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html

In [ ]:
df_ = pd.DataFrame([[4, 9]] * 3, columns=['A', 'B'])
df_.head()

In [ ]:
soma_um = lambda x: x + 1
df_['A1'] = df_['A'].apply(soma_um)
df_.head()

In [ ]:
#df['data_entrada_brasil'].apply(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y'))

##### filtrando antes de aplicar

In [ ]:
df[df['data_entrada_brasil'] == 'Năo informado']

In [ ]:
df['data_entrada_brasil_formatado'] = \
    df[df['data_entrada_brasil'] != 'Năo informado']['data_entrada_brasil'].apply(
        lambda x: datetime.datetime.strptime(x, '%d/%m/%Y')
    )
df.head()

In [ ]:
df.info()

#### Visualizando frequência de imigração por ano

In [ ]:
df['ano_entrada'] = df['data_entrada_brasil_formatado'].apply(lambda x: x.year)
df['ano_entrada'].unique()

In [ ]:
plt.figure(figsize=(20, 10))
ax = sns.countplot(x='ano_entrada', data=df)
ax.set_xticklabels(rotation=90, labels=ax.get_xticklabels())
ax

In [ ]:
plt.figure(figsize=(20, 10))
ax = sns.countplot(x='ano_entrada', data=df[df['ano_entrada'].between(2008,2020)])

##### Observamos que existe um crescimento, porém... O que aconteceu em 2016?
- Vamos observar agora por pais de origem nesse ano... Qual seria a origem mais provável pelo aumento?

In [ ]:
plt.figure(figsize=(20, 10))
ax = sns.countplot(x='pais_origem', data=df[df['ano_entrada'] == 2016])
ax.set_xticklabels(rotation=90, labels=ax.get_xticklabels())
ax

#### Vamos observar o aumento só de imigrantes de lá

In [ ]:
plt.figure(figsize=(20, 10))
ax = sns.countplot(x='ano_entrada', data=df[df['pais_origem'] == 'Angola'])
ax.set_xticklabels(rotation=90, labels=ax.get_xticklabels())
ax

![alt text](angola.png)

##### E se olharmos também no tempo Venezuelanos?

In [ ]:
plt.figure(figsize=(20, 10))
ax = sns.countplot(x='ano_entrada', data=df[df['pais_origem'] == 'Venezuela'])
ax.set_xticklabels(rotation=90, labels=ax.get_xticklabels())
ax

<div class="alert alert-block alert-info">
    <h2>Categorizando Dados</h2>
</div>

![alt text](tipos_variaveis.jpg)

In [ ]:
df['fx_etaria'].nunique() #Observando a variação

In [ ]:
df['fx_etaria'].unique() #Observando valores

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 6))
ax = sns.countplot(x='fx_etaria', data=df)

#### E se eu categorizar com ordem

In [ ]:
df['fx_etaria'] = df['fx_etaria'].astype('category')
df['fx_etaria'].cat.reorder_categories(['0 a 11 anos',
                                         '12 a 17 anos',
                                         '20 a 29 anos',
                                         '18 a 29 anos',
                                         '30 a 39 anos',
                                         '40 a 49 anos',
                                         '50 a 59 anos',
                                         'Maiores de 60 anos',  
                                          'Năo informado'], inplace=True)
plt.figure(figsize=(16, 6))
ax = sns.countplot(x='fx_etaria', data=df)

#### Peraí, tem grupo de intersecção de idade
- Vamos então olhar as pessoas dessa faixa

In [ ]:
df[df['fx_etaria'] == '20 a 29 anos']

#### O que fazer?
- Decidimos por unir com a faixa mais próxima
- O correto https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iat.html

In [ ]:
df.loc[df['fx_etaria'] == '20 a 29 anos', 'fx_etaria'] = '18 a 29 anos'
df[df['fx_etaria'] == '20 a 29 anos']

<div class="alert alert-block alert-info">
    <h2>"DesCategorizando" Dados</h2>
</div>
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html

In [ ]:
df_corr = df[['pais_origem', 'cor_raca']]

df_corr = df_corr[(df_corr['pais_origem'] == 'Angola') | (df_corr['pais_origem'] == 'Venezuela')]

df_corr = pd.get_dummies(df_corr, columns=['pais_origem', 'cor_raca'])
df_corr.tail()

![alt text](correlacao.jpg)

In [ ]:
corr = df_corr.corr()
corr.style.background_gradient(cmap='coolwarm')

<div class="alert alert-block alert-info">
    <h2>Amostragem</h2>
</div>

### Aleatória

In [ ]:
df_ale = df.sample(n=100)
df_ale.head()

In [ ]:
df_ale = df.sample(frac=0.1)
df_ale.shape

### Sistemática

In [ ]:
semente = 5
indices = np.arange(0, 100, semente)
indices

In [ ]:
df.loc[indices,:].head()

<div class="alert alert-block alert-info">
    <h2>Missing Values</h2>
</div>

https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html  

https://pandas.pydata.org/pandas-docs/version/0.25.0/reference/api/pandas.DataFrame.fillna.html#pandas.DataFrame.fillna  

In [ ]:
df['dif_dtcad_dtingr_meses'].unique()

In [ ]:
df_dif = df[
    (df['dif_dtcad_dtingr_meses'] != 'Năo informado') & 
    (df['dif_dtcad_dtingr_meses'] != 'Incongruencia') &
    (df['dif_dtcad_dtingr_meses'] != '#NÚM!')
]
df_dif['dif_dtcad_dtingr_meses'] = df_dif['dif_dtcad_dtingr_meses'].astype('int')

sns.set(style='whitegrid')
ax = sns.boxplot(data=df_dif, x='dif_dtcad_dtingr_meses')

![alt text](boxplot.png)

In [ ]:
moda = df_dif['dif_dtcad_dtingr_meses'].mode()
moda

In [ ]:
media = df_dif['dif_dtcad_dtingr_meses'].mean()
media

In [ ]:
mediana = df_dif['dif_dtcad_dtingr_meses'].median()
mediana

In [ ]:
df.loc[df['dif_dtcad_dtingr_meses'] == 'Năo informado', 'dif_dtcad_dtingr_meses'] = '0'
df.loc[df['dif_dtcad_dtingr_meses'] == 'Incongruencia', 'dif_dtcad_dtingr_meses'] = '0'
df.loc[df['dif_dtcad_dtingr_meses'] == '#NÚM!', 'dif_dtcad_dtingr_meses'] = '0'

df['dif_dtcad_dtingr_meses'] = df['dif_dtcad_dtingr_meses'].astype('int')
df.describe()

In [ ]:
df_dif.describe()

<div class="alert alert-block alert-info">
    <h2>Data Profile</h2>
</div>

In [ ]:
from pandas_profiling import ProfileReport

df = pd.read_csv('bancocrai2014a2019---sistematizacao-geoinfo-atualizada.csv', sep=';', encoding='iso8859_2')

profile = ProfileReport(df)
profile

In [ ]:
profile.to_file(output_file="profile.html")